# Django REST Framework:
- Django REST framework is another popular option for building REST APIs.
- It adds the REST API functionality on top of the existing Django project.
- To use Django REST framework, we need a Django project to work with.

## Step 1: Install and setup Django and Django REST API:
- First, install ``Django`` and ``djangorestframework`` with pip:
```sh
python -m pip install Django djangorestframework
```
- We can now use the ``django-admin`` tool to create a new Django project:
```sh
django-admin startproject countryapi
```
- This command creates a new folder in your current directory called ``countryapi``. Inside this folder are all the files you need to run your Django project.
- Next, you’re going to create a new **Django application** inside your project. Django breaks up the functionality of a project into **applications**. Each application manages a distinct part of the project.
- To create the application, change directories to ``countryapi`` and run the following command:
```sh
python manage.py startapp countries
```
- This creates a new ``countries`` folder inside your project. Inside this folder are the base files for this application.
- Once the application is created, we need to tell Django about it. 
- Alongside the countries folder that we created, is another folder called ``countryapi``. This folder contains **configurations** and **settings** for your project.
- **Note:** This folder has the same name as the root folder that Django created when we ran ``django-admin startproject countryapi``.
- We need to add the following lines to ``INSTALLED_APPS`` to tell Django about the ``countries`` application and Django REST framework (adding ``rest_framework`` and ``countries`` in the list):
```python
# countryapi/settings.py
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "rest_framework",
    "countries",
]
```
- Django plugins are Django applications that are packaged up and distributed and that anyone can use and ``rest_framework`` is another such plugin, thus we need to add it before use.

## Step 2: Create a Django Model:
- We need to create a Django model to define the fields of our data. Inside of the countries application, update ``models.py`` with the following code:
```python
# countries/models.py
from django.db import models

class Country(models.Model):
    name = models.CharField(max_length=100)
    capital = models.CharField(max_length=100)
    area = models.IntegerField(help_text="(in square kilometers)")
```
- Django will use this model to create the database table and columns for the country data.
- To have Django update the database based on this model:
```
> python manage.py makemigrations
Migrations for 'countries':
  countries/migrations/0001_initial.py
    - Create model Country

> python manage.py migrate
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, countries, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  ...
```
- These commands use Django migrations to create a new table in the database.
- The table is initially empty. But it would be good to have some initial data so that we can test our Django REST framework.
- We use **[Django fixture](https://realpython.com/django-pytest-fixtures/)** for this.
- We need to create a file ``countries.json`` and have the below data:
```json
[
    {
        "model": "countries.country",
        "pk": 1,
        "fields": {
            "name": "Thailand",
            "capital": "Bangkok",
            "area": 513120
        }
    },
    {
        "model": "countries.country",
        "pk": 2,
        "fields": {
            "name": "Australia",
            "capital": "Canberra",
            "area": 7617930
        }
    },
    {
        "model": "countries.country",
        "pk": 3,
        "fields": {
            "name": "Egypt",
            "capital": "Cairo",
            "area": 1010408
        }
    }
]
```
- This JSON contains database entries for three countries. 
- Calling the following command will load this data in the database and adds 3 rows in the DB:
```
> python manage.py loaddata countries.json
Installed 3 object(s) from 1 fixture(s)
```
### Model Serializers:
- Django REST Framework takes an existing Django model and converts it to JSON for REST API.
- It does this with **model serializers** which tells Django REST framework how to convert a model instance to JSON and what data to include. 
- We need to create a serializer for the ``Country`` mode. We will start with creating a file, ``serializers.py`` inside the ``countries`` app with the following code:
```python
# countries/serializers.py
from rest_framework import serializers
from .models import Country

class CountrySerializer(serializers.ModelSerializer):
    class Meta:
        model = Country
        fields = ["id", "name", "capital", "area"]
```
- This serializer: ``CountrySerializer()``, is a subclass of class: ``serializer.ModelSerializer`` which automatically generates JSON content based on the model fields of ``Country``. 
- Unless specified, a ModelSerializer subclass includes all fields from the Django model in the JSON. 
- Hence, we must specify the list of data we wish to include by setting the ``fields``variable in order to modify this behaviour.

## Step 3: Create Django views:
- As we know, Django uses *views* to query data from the database and display to the user.
- Similarly, Django REST framework also makes use of *views*.
- However, instead of writing REST API views from scratch, we can subclass Django REST framework’s ``ModelViewSet`` class, which has default views for common REST API operations.
- **Note:** The Django REST framework documentation refers to these views as [actions](https://www.django-rest-framework.org/api-guide/viewsets/#viewset-actions).
- Here’s a list of the actions that ``ModelViewSet`` provides and their equivalent HTTP methods:

| HTTP Method | Action | Description |
| :---------- | :----- | :---------- |
| GET | .list() | Get a list of countries. |
| GET | .retrieve() | Get a single country. |
| POST | .create() | Create a new country. |
| PUT | .update() | Update a country. |
| PATCH | .partial_update() | Partially update a country. |
| DELETE | .destroy() | Delete a country. |

- These actions map to the standard HTTP methods as expected in a REST API. 
- We can override these actions in our subclass or add additional actions based on the requirements.
- Below is the code for a ``ModelViewSet`` subclass called ``CountryViewSet``.
- This class will generate the views needed to manage ``country`` data.
- Just like in case of the ``CountrySerializer`` class, we need to modify the code in ``views.py`` inside the ``countries.py`` file:
```python
# countries/views.py
from rest_framework import viewsets

from .models import Country
from .serializers import CountrySerializer

class CountryViewSet(viewsets.ModelViewSet):
    serializer_class = CountrySerializer
    queryset = Country.objects.all()
```
- ``serializer_class = CountrySerializer`` tells Django REST framework which serializer to use.
- ``queryset = Country.objects.all()`` tells how to query the DB for this specific set of views.

## Step 4: Update the urls.py locally (for the app):
- Once the views are created, we need to map them to the respective endpoints or URLs.
- Django REST framework provides a ``DefaultRouter`` that will automatically generate URLs for a ``ModelViewSet``:
- We need to Create a ``urls.py`` file in the ``countries`` application and add the following code to the file:
```python
# countries/urls.py
from django.urls import path, include
from rest_framework.routers import DefaultRouter

from .views import CountryViewSet

router = DefaultRouter()
router.register(r"countries", CountryViewSet)

urlpatterns = [
    path("", include(router.urls))
]
```
- This code creates a ``DefaultRouter`` and registers ``CountryViewSet`` under the ``countries`` URL.
- This will place all the URLs for ``CountryViewSet`` under ``/countries/``.
- **Note:** Django REST framework automatically appends a forward slash (/) to the end of any endpoints generated by DefaultRouter. We can disable this behavior like so:
```python
router = DefaultRouter(trailing_slash=False)
```
- This will disable the forward slash at the end of endpoints.

## Step 5: Update the urls.py globally (for the project):
- Finally, we also need to update the project's base ``urls.py`` file to include all the ``countries`` URLs in the project.
- Update the ``urls.py``inside the ``countryapi`` folder:
```python
# countryapi/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("countries.urls")),
]
```

## Step 6: Run the application:
- Run the following command in the root ``countryapi`` directory to start the Django development server:
```sh
python manage.py runserver
```
- Once the dev server is running, we can copy the link and send a ``GET`` request to ``/countries/``.
```sh
curl -i http://127.0.0.1:8000/countries/ -w '\n'
```
- Django REST framework sends back a JSON response with the three countries we added earlier:
```json
HTTP/1.1 200 OK
...

[
    {
        "id": 1,
        "name":"Thailand",
        "capital":"Bangkok",
        "area":513120
    },
    {
        "id": 2,
        "name":"Australia",
        "capital":"Canberra",
        "area":7617930
    },
    {
        "id": 3,
        "name":"Egypt",
        "capital":"Cairo",
        "area":1010408
    }
]
```
- The ``DefaultRouter`` we created in ``countries/urls.py`` provides URLs for requests to all the standard API endpoints:
  - GET ``/countries/``
  - GET ``/countries/<country_id>/``
  - POST ``/countries/``
  - PUT ``/countries/<country_id>/``
  - PATCH ``/countries/<country_id>/``
  - DELETE ``/countries/<country_id>/``
- ``POST`` request to ``/countries/`` to a create a new ``Country`` in the Django project:
```sh
curl -i http://127.0.0.1:8000/countries/ \
-X POST \
-H 'Content-Type: application/json' \
-d '{"name":"Germany", "capital": "Berlin", "area": 357022}' \
-w '\n'
```
- This creates a new ``Country`` with the JSON we sent in the request. 
- Django REST framework returns a ``201 Created`` status code and the new ``Country``:
```json
HTTP/1.1 201 Created
...

{
    "id":4,
    "name":"Germany",
    "capital":"Berlin",
    "area":357022
}
```
- **Note:** By default, the response doesn’t include a new line at the end. This means that the JSON may run into your command prompt. The curl command above includes ``-w '\n'`` to add a newline character after the JSON to fix this issue.
- You can view an existing ``Country`` by sending a request to ``GET /countries/<country_id>/`` with an existing id. 
- Run the following command to get the first ``Country``:
```sh
curl -i http://127.0.0.1:8000/countries/1/ -w '\n'
```
- The response contains the information for the first ``Country``:
```json
HTTP/1.1 200 OK
...

{
    "id":1,
    "name":"Thailand",
    "capital":"Bangkok",
    "area":513120
}
```
- Django REST framework is a great option for building REST APIs, especially if you have **an existing Django project** and you want to add an API.

--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------